**Tahapan latihan kali ini adalah:**


- Memastikan TensorFlow yang digunakan di Google Colab adalah versi di atas 2.0.

- Mengunduh dataset dan melakukan extract file dengan metode unzip.

- Menampung direktori setiap kelas pada direktori train dan validasi ke dalam variabel.

- Pre-processing data dengan image augmentation.

- Mempersiapkan data latih yang akan dipelajari oleh model.

- Membangun arsitektur model dengan Convolutional Neural Network (CNN).

- Compile dan latih model dengan model.compile dan model.fit hingga mendapatkan akurasi yang diinginkan.

- Menguji model yang telah dibuat dengan menggunakan gambar yang belum dikenali oleh model.



In [1]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [2]:
!wget --no-check-certificate \
  https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip \
  -O /tmp/messy_vs_clean_room.zip

--2024-02-09 15:58:17--  https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip
Resolving dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com (dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com)... 104.18.12.192, 104.18.13.192, 2606:4700::6812:cc0, ...
Connecting to dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com (dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com)|104.18.12.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com attempted to set domain to digitaloceanspaces.com
Length: 70392746 (67M) [application/zip]
Saving to: ‘/tmp/messy_vs_clean_room.zip’

/tmp/messy_vs_clean 100%[===================>]  67.13M  77.4MB/s    in 0.9s    

2024-02-09 15:58:18 (77.4 MB/s) - ‘/tmp/messy_vs_clean_room.zip’ saved [70392746/70392746]



In [3]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/tmp/messy_vs_clean_room.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [4]:
os.listdir('/tmp/images/train')


['clean', 'messy']

In [5]:
os.listdir('/tmp/images/val')


['clean', 'messy']

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [10]:
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [11]:
# latih model dengan model.fit
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=20, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/20
25/25 - 19s - loss: 0.7402 - accuracy: 0.5100 - val_loss: 0.6923 - val_accuracy: 0.5000 - 19s/epoch - 764ms/step
Epoch 2/20
25/25 - 13s - loss: 0.6961 - accuracy: 0.5200 - val_loss: 0.6743 - val_accuracy: 0.5500 - 13s/epoch - 534ms/step
Epoch 3/20
25/25 - 13s - loss: 0.6843 - accuracy: 0.5400 - val_loss: 0.6070 - val_accuracy: 0.5500 - 13s/epoch - 533ms/step
Epoch 4/20
25/25 - 13s - loss: 0.6777 - accuracy: 0.7000 - val_loss: 0.5900 - val_accuracy: 0.8000 - 13s/epoch - 537ms/step
Epoch 5/20
25/25 - 14s - loss: 0.7451 - accuracy: 0.5700 - val_loss: 0.6742 - val_accuracy: 0.5000 - 14s/epoch - 541ms/step
Epoch 6/20
25/25 - 13s - loss: 0.6880 - accuracy: 0.5500 - val_loss: 0.6643 - val_accuracy: 0.5500 - 13s/epoch - 530ms/step
Epoch 7/20
25/25 - 13s - loss: 0.6450 - accuracy: 0.7000 - val_loss: 0.4630 - val_accuracy: 0.8000 - 13s/epoch - 535ms/step
Epoch 8/20
25/25 - 13s - loss: 0.5217 - accuracy: 0.7400 - val_loss: 0.4552 - val_accuracy: 0.7000 - 13s/epoch - 535ms/step
Epoch 9/

In [12]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))


Saving download.jpg to download.jpg


In [13]:
print(train_generator.class_indices)

{'clean': 0, 'messy': 1}
